In [3]:
import pandas as pd
import json
# Unicode Handling
from __future__ import unicode_literals

import pandas as pd
import json
import numpy as np

## Load spacy

from spacy.en import English
nlp_toolkit = English()
nlp_toolkit

data = pd.read_csv("assets/dataset/stumbleupon.tsv", sep='\t')
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


 ### DEMO: LDA IN GENSIM

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=False,
                     stop_words='english',
                     min_df=3)

In [5]:
docs = cv.fit_transform(data.body.dropna())

In [6]:
# Build a mapping of numerical ID to word
id2word = dict(enumerate(cv.get_feature_names()))

In [8]:
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus

In [9]:
# First we convert our word-matrix into gensim's format
corpus = Sparse2Corpus(docs, documents_columns = False)

In [10]:
# Then we fit an LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=15)

In [11]:
num_topics = 15
num_words_per_topic = 8
for ti, topic in enumerate(lda_model.show_topics(num_topics = num_topics, num_words = num_words_per_topic)):
    print("Topic: %d" % (ti))
    print (topic)
    print()

Topic: 0
(0, u'0.012*"food" + 0.011*"raw" + 0.010*"body" + 0.007*"weight" + 0.007*"healthy" + 0.007*"foods" + 0.006*"health" + 0.006*"fat"')
()
Topic: 1
(1, u'0.027*"com" + 0.022*"http" + 0.020*"www" + 0.013*"2011" + 0.013*"2009" + 0.013*"online" + 0.013*"2010" + 0.010*"10"')
()
Topic: 2
(2, u'0.008*"like" + 0.005*"dress" + 0.005*"look" + 0.004*"just" + 0.004*"love" + 0.004*"new" + 0.004*"fashion" + 0.004*"make"')
()
Topic: 3
(3, u'0.017*"chocolate" + 0.012*"butter" + 0.011*"cup" + 0.011*"sugar" + 0.009*"cake" + 0.009*"baking" + 0.008*"recipe" + 0.008*"make"')
()
Topic: 4
(4, u'0.010*"cup" + 0.009*"add" + 0.008*"minutes" + 0.007*"recipe" + 0.006*"chicken" + 0.006*"cheese" + 0.006*"salt" + 0.006*"heat"')
()
Topic: 5
(5, u'0.009*"health" + 0.005*"brain" + 0.005*"study" + 0.005*"people" + 0.004*"cancer" + 0.003*"help" + 0.003*"natural" + 0.003*"research"')
()
Topic: 6
(6, u'0.008*"olympics" + 0.007*"olympic" + 0.007*"world" + 0.005*"calories" + 0.005*"2012" + 0.004*"gold" + 0.004*"medal" 

 ### DEMO: Word2Vec IN GENSIM

In [12]:
# Setup the body text
text = data.body.dropna().map(lambda x: x.split())

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(text, size=100, window=5, min_count=5, workers=4)

/Users/patrickrhbrown/anaconda/lib/python2.7/site-packages/gensim-2.3.0-py2.7-macosx-10.7-x86_64.egg/gensim/models/word2vec.py:813: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


In [ ]:
model.most_similar(positive=['cookie', 'brownie'])
